In [5]:
import pandas as pd

df = pd.read_excel("/media/mrk/Data/E-JUST/publication/Exp_speed/Exp_speed_new/data_set.xlsx", sheet_name='test_set')
df.head()

,Disturbance Y/M/D (UT),CME start_time,Vp,Vlat,Vfrt,Vsw,aw,2aw,Tobs,V1,V2,V3,V4,T1,T2,T3,T4,Tdbm,Vdbm
0,2000/01/22 0023,2000/01/18 1754,570,266,85.5,400,39.5,80,78.48,544,377,338,514,76.60,110.57,123.25,81.12,77.64,454.0
1,2000/02/11 0258,2000/02/08 0930,630,294,94.5,510,50.5,102,65.47,611,442,401,629,68.17,94.31,103.92,66.27,62.49,571.0
2,2000/02/11 2352,2000/02/10 0230,810,378,121.5,590,58.5,118,45.37,813,623,563,828,51.25,66.93,74.07,50.31,50.52,685.0
3,2000/02/14 0731,2000/02/12 0431,780,364,117.0,680,67.5,136,51.00,777,597,545,854,53.61,69.85,76.43,48.81,46.42,766.0
4,2001/01/23 1048,2001/01/20 2130,600,280,90.0,550,54.5,110,61.30,575,413,378,625,72.40,100.90,110.19,66.67,60.75,598.0


In [106]:
start_time = df[df.columns[1]].values
end_time = df[df.columns[0]].values
print(end_time[:5])

['2000/01/22 0023' '2000/02/11 0258' '2000/02/11 2352' '2000/02/14 0731'
 '2001/01/23 1048']


In [107]:
# Estimating the Transit time of ICME/shock based upon date-time

from datetime import datetime

date_time_format = "%Y/%m/%d %H%M"

transit_time = []
for j in range(len(end_time)):
    start = start_time[j].strip()  
    end = end_time[j].strip()

    try:
        date_time1 = datetime.strptime(start, date_time_format)
        date_time2 = datetime.strptime(end, date_time_format)
    except ValueError as e:
        print(f"Error parsing date-time: {e}")
        continue  

    time_difference = date_time2 - date_time1
    total_hours = time_difference.total_seconds() / 3600
    transit_time.append(round(total_hours,2))

print(transit_time[:6])

# print(len(transit_time))

# df['Time'] = transit_time

# df.to_excel("C:/Users/MrK/Desktop/time.xlsx", index=False)

[78.48, 65.47, 45.37, 51.0, 61.3, 68.52]
50


In [32]:
# Importing test dataset

import tensorflow as tf
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
df = pd.read_excel("/media/mrk/Data/E-JUST/publication/Exp_speed/Exp_speed_new/data_set.xlsx", sheet_name='FNN_test')
df.head()

,Disturbance Y/M/D (UT),CME start_time,Vt,Vlat,Vfrt,Vsw,2aw,Tobs
0,2000/01/22 0023,2000/01/18 1754,530.920829,266,85.5,400,80,78.48
1,2000/02/11 0258,2000/02/08 0930,636.423807,294,94.5,510,102,65.47
2,2000/02/11 2352,2000/02/10 0230,918.374844,378,121.5,590,118,45.37
3,2000/02/14 0731,2000/02/12 0431,816.993464,364,117.0,680,136,51.00
4,2001/01/23 1048,2001/01/20 2130,679.717238,280,90.0,550,110,61.30


In [36]:
# Splitting data into X-y test

X_test = df[df.columns[2:-1]].values
y_test = df[df.columns[-1]].values.reshape(-1,1)

X_test = tf.constant(X_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

sc_x = StandardScaler()
sc_y = StandardScaler()

X_test = sc_x.fit_transform(X_test)
y_test = sc_y.fit_transform(y_test)
print(X_test[:5])
print()
print(y_test[:5])

[[-0.62541888 -0.62571463 -0.92353672 -1.12194564 -1.12194564]
 [-0.04884055 -0.19964436 -0.43646784 -0.16077726 -0.16077726]
 [ 1.49203357  1.07856646  1.02473881  0.53825429  0.53825429]
 [ 0.9379802   0.86553132  0.78120437  1.32466479  1.32466479]
 [ 0.18775969 -0.41267949 -0.68000228  0.18873852  0.18873852]]

[[ 0.36310731]
 [-0.23266843]
 [-1.15312128]
 [-0.89530287]
 [-0.42362812]]


In [31]:
# loading the saved model

import tensorflow as tf

model = model = tf.keras.models.load_model('/media/mrk/Data/E-JUST/publication/Exp_speed/Exp_speed_new/model/Exp_speed_new.keras')



In [38]:
# Predicting Test dataset with neural network model

import numpy as np
import random

random.seed(1234)
y_pred = model.predict(X_test)


print('R2 score:', r2_score(y_test, y_pred))
print('mean absolute error =', np.mean(np.abs(sc_y.inverse_transform(y_pred)- sc_y.inverse_transform(y_test))))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
R2 score: 0.943153084543929
mean absolute error = 3.8392459106445314


In [20]:
import pandas as pd

df = pd.read_excel("/media/mrk/Data/E-JUST/publication/Exp_speed/Exp_speed_new/test_set.xlsx")
df.head()

,Disturbance Y/M/D (UT),CME start_time,Vp,Vlat,Vfrt,Vsw,2aw,Tobs,V1,V2,V3,V4,T1,T2,T3,T4,Tfnn,Tdbm,Vdbm
0,2000/01/22 0023,2000/01/18 1754,570,266,85.5,400,80,78.48,544,504,505,496,76.60,82.68,82.46,83.97,85.187111,80.64,449
1,2000/02/11 0258,2000/02/08 0930,630,294,94.5,510,102,65.47,607,612,604,603,68.64,68.11,69.02,69.06,63.715534,65.03,563
2,2000/02/11 2352,2000/02/10 0230,810,378,121.5,590,118,45.37,813,814,807,803,51.25,51.21,51.64,51.92,44.094547,52.07,679
3,2000/02/14 0731,2000/02/12 0431,780,364,117.0,680,136,51.00,777,835,816,824,53.61,49.93,51.09,50.56,48.806149,48.07,757
4,2001/01/23 1048,2001/01/20 2130,600,280,90.0,550,110,61.30,575,609,595,601,72.40,68.41,70.07,69.32,58.823994,63.53,586


In [23]:
# calculating the mean absoulte error for RGBM, FFNN and DBM 

import numpy as np

t_obs = df['Tobs']
t1 = df['T1']
t2 = df['T2']
t3 = df['T3']
t4 = df['T4']
t_dbm = df['Tdbm']
t_fnn = df['Tfnn']

mae_t1 = np.mean(np.abs(t1-t_obs))
mae_t2 = np.mean(np.abs(t2-t_obs))
mae_t3 = np.mean(np.abs(t3-t_obs))
mae_t4 = np.mean(np.abs(t4-t_obs))
mae_t_dbm = np.mean(np.abs(t_dbm-t_obs))
mae_t_fnn = np.mean(np.abs(t_fnn-t_obs))


print(f"MAE for Time 1 = {round(mae_t1, 2)}")
print(f"MAE for Time 2 = {round(mae_t2, 2)}")
print(f"MAE for Time 3 = {round(mae_t3, 2)}")
print(f"MAE for Time 4 = {round(mae_t4, 2)}")
print(f"MAE for DBM = {round(mae_t_dbm, 2)}")
print(f"MAE for FNN = {round(mae_t_fnn, 2)}")

MAE for Time 1 = 10.29
MAE for Time 2 = 8.54
MAE for Time 3 = 8.69
MAE for Time 4 = 8.42
MAE for DBM = 8.29
MAE for FNN = 3.84
